In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt

df=pd.read_csv('/Users/xueyuanli/Desktop/DS-team/time_series_ML_features_final.csv', parse_dates=['weeks_monday','month_year'])
df


,phone_model,weeks_monday,month,year,month_year,days_since_release_date,claim
0,apple iphone 11,2021-07-05,7,2021,2021-07-01,654,2047
1,apple iphone x,2021-07-05,7,2021,2021-07-01,1340,555
2,apple iphone 11,2021-07-12,7,2021,2021-07-01,661,1900
3,apple iphone x,2021-07-12,7,2021,2021-07-01,1347,496
4,apple iphone 11,2021-07-19,7,2021,2021-07-01,668,1918
...,...,...,...,...,...,...,...
321,apple iphone 14,2023-02-06,2,2023,2023-02-01,143,61
322,apple iphone 12,2023-02-06,2,2023,2023-02-01,836,1456
323,apple iphone 13,2023-02-06,2,2023,2023-02-01,470,1191
324,apple iphone 11,2023-02-06,2,2023,2023-02-01,1235,1307


In [4]:
iphone_12= df[df['phone_model'].str.startswith('apple iphone 12')]
iphone12_df = iphone_12[['weeks_monday', 'claim']]

iphone_13= df[df['phone_model'].str.startswith('apple iphone 13')]
iphone13_df = iphone_13[['weeks_monday', 'claim']]

iphone_14= df[df['phone_model'].str.startswith('apple iphone 14')]
iphone14_df = iphone_14[['weeks_monday', 'claim']]

,weeks_monday,claim
9,2021-08-02,69
12,2021-08-09,219
15,2021-08-16,224
18,2021-08-23,292
21,2021-08-30,274
...,...,...
302,2023-01-09,1558
307,2023-01-16,1552
311,2023-01-23,1407
317,2023-01-30,1171


In [19]:
def prepare_data(df):
    iphone12_timeshift = df.copy(deep = True)
    iphone12_timeshift['1_week_ago'] = iphone12_timeshift['claim'].shift(1)
    iphone12_timeshift['2_week_ago'] = iphone12_timeshift['1_week_ago'].shift(1)
    iphone12_timeshift['3_week_ago'] = iphone12_timeshift['2_week_ago'].shift(1)
    iphone12_timeshift = iphone12_timeshift.rename(columns={'claim': 'this_week'})
    iphone12_timeshift = iphone12_timeshift.drop(iphone12_timeshift.columns[0], axis=1)
    iphone12_timeshift =  iphone12_timeshift.dropna()
    iphone12_timeshift = iphone12_timeshift.reset_index(drop=True)
    
    return iphone12_timeshift


In [29]:
def split_train_data(train_data):
    X_train = train_data[['1_week_ago', '2_week_ago', '3_week_ago']]
    y_train = train_data['this_week']
    return X_train, y_train

def split_test_data(test_data):
    X_test = test_data[['1_week_ago', '2_week_ago', '3_week_ago']]
    y_test = test_data['this_week']
    return X_test,y_test


In [169]:
def wmape(actual, forecast):
    # we take two series and calculate an output a wmape from it, not to be used in a grouping function

    # make a series called mape
    se_mape = abs(actual-forecast)/actual

    # get a float of the sum of the actual
    ft_actual_sum = actual.sum()

    # get a series of the multiple of the actual & the mape
    se_actual_prod_mape = actual * se_mape

    # summate the prod of the actual and the mape
    ft_actual_prod_mape_sum = se_actual_prod_mape.sum()

    # float: wmape of forecast
    ft_wmape_forecast = ft_actual_prod_mape_sum / ft_actual_sum

    # return a float
    return ft_wmape_forecast



# 1, iphone12-> iphone13

## Train iPhone 12 until end of Feb 2022

## Validate iPhone 12 March 2022

## Test iPhone 13 March 2022

In [156]:
iphone12_train = iphone12_df[(iphone12_df['weeks_monday'] <= '2022-02-28')] 
iphone12_train=prepare_data(iphone12_train)
iphone12_validate = iphone12_df[(iphone12_df['weeks_monday'] >= '2022-03-07') & (iphone12_df['weeks_monday'] <= '2022-03-28')].reset_index()
iphone12_validate=prepare_data(iphone12_validate)
iphone13_test = iphone13_df[(iphone13_df['weeks_monday'] >= '2022-03-07') & (iphone13_df['weeks_monday'] <= '2022-03-28')].reset_index()
iphone13_test = prepare_data(iphone13_test)


X_train, y_train= split_train_data(iphone12_train)
X_validate, y_validate = split_test_data(iphone12_validate)
X_test, y_test= split_test_data(iphone13_test)

In [171]:
print(iphone13_test)

  weeks_monday  this_week  1_week_ago  2_week_ago  3_week_ago
0   2022-03-28         43       24.00       31.00       32.00


# XGBoost

In [157]:
import xgboost as xgb
xgb_model=xgb.XGBRegressor()
xgb_model=xgb_model.fit(X_train, y_train)
pred=xgb_model.predict(X_validate)


In [158]:
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_validate, pred)
print('MSE:', mse)

wmape=wmape(y_validate, pred)

print("WMAPE: {:.2%}".format(wmape))

MSE: 71023.90912834182
WMAPE: 37.75%


# AdaBoostRegressor

In [139]:
from sklearn import ensemble

from sklearn.ensemble import AdaBoostRegressor

AB_model=AdaBoostRegressor()
AB_model.fit(X_train, y_train)

pred = AB_model.predict(X_validate)


In [115]:
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_validate, pred)
print('MSE:', mse)

wmape=wmape(y_validate, pred)
print("WMAPE: {:.2%}".format(wmape))

MSE: 50700.02777777779
WMAPE: 31.89%


In [131]:
print(X_test)
print(y_test)

   1_week_ago  2_week_ago  3_week_ago
0       24.00       31.00       32.00
0    43
Name: this_week, dtype: int64


In [140]:
pred = AB_model.predict(X_test)
pred *= avg_growth_decimal

print(pred)

mse = mean_squared_error(y_test, pred)
print('MSE:', mse)

wmape=wmape(y_test, pred)
print("WMAPE: {:.2%}".format(wmape))

[297.5148956]
MSE: 64777.83208203697
WMAPE: 591.90%


# BaggingRegressor

In [116]:
from sklearn.ensemble import BaggingRegressor

BR_model=BaggingRegressor()
BR_model.fit(X_train, y_train)

pred = BR_model.predict(X_validate)

In [118]:
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_validate, pred)
print('MSE:', mse)

wmape=wmape(y_validate, pred)
print("WMAPE: {:.2%}".format(wmape))

MSE: 45454.24
WMAPE: 30.20%


# DecisionTreeRegressor

In [119]:
from sklearn.tree import DecisionTreeRegressor

DT_model=DecisionTreeRegressor()
DT_model.fit(X_train, y_train)

pred = DT_model.predict(X_validate)

In [121]:
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_validate, pred)
print('MSE:', mse)

wmape=wmape(y_validate, pred)
print("WMAPE: {:.2%}".format(wmape))

MSE: 35721.0
WMAPE: 26.77%


In [122]:
print(y_test)
print(X_test)

0    43
Name: this_week, dtype: int64
   1_week_ago  2_week_ago  3_week_ago
0       24.00       31.00       32.00


In [124]:
dt_pred = DT_model.predict(X_test)
print(dt_pred)
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_test, dt_pred)
print('MSE:', mse)

wmape=wmape(y_test, dt_pred)
print("WMAPE: {:.2%}".format(wmape))

[292.]
MSE: 62001.0
WMAPE: 579.07%


# RandomForestRegressor

In [85]:
from sklearn.ensemble import RandomForestRegressor

RFR_model=RandomForestRegressor()
RFR_model.fit(X_train, y_train)

pred = RFR_model.predict(X_validate)

In [87]:
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_validate, pred)
print('MSE:', mse)

wmape=wmape(y_validate, pred)
print("WMAPE: {:.2%}".format(wmape))

MSE: 51510.84159999999
WMAPE: 32.15%


# LinearRegression

In [142]:
from sklearn.linear_model import LinearRegression

lin_model = LinearRegression()
lin_model.fit(X_train, y_train)

pred = lin_model.predict(X_validate)

In [90]:
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_validate, pred)
print('MSE:', mse)

wmape=wmape(y_validate, pred)
print("WMAPE: {:.2%}".format(wmape))

MSE: 8607.41076136259
WMAPE: 13.14%


In [143]:
lin_pred = lin_model.predict(X_test)

from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_test, lin_pred)
print('MSE:', mse)

wmape=wmape(y_test, lin_pred)
print("WMAPE: {:.2%}".format(wmape))

MSE: 1943.084219882915
WMAPE: 102.51%


# 2, iphone13-> iphone14

# Train iPhone 13 until end of Dec 2022

# Validate iPhone 13 Jan 2023

# Test iPhone 14 Jan 2023

In [160]:
iphone13_train = iphone13_df[(iphone13_df['weeks_monday'] < '2023-01-02')] 
iphone13_train=prepare_data(iphone13_train)
iphone13_validate = iphone13_df[(iphone13_df['weeks_monday'] >= '2023-01-02') & (iphone13_df['weeks_monday'] <= '2023-02-06')].reset_index()
iphone13_validate=prepare_data(iphone13_validate)
iphone14_test = iphone14_df[(iphone14_df['weeks_monday'] >= '2023-01-02') & (iphone14_df['weeks_monday'] <= '2023-02-06')].reset_index()
iphone14_test = prepare_data(iphone14_test)


X_train, y_train= split_train_data(iphone13_train)
X_validate, y_validate = split_test_data(iphone13_validate)
X_test, y_test= split_test_data(iphone14_test)

# xgboost

In [161]:
import xgboost as xgb
xgb_model=xgb.XGBRegressor()
xgb_model=xgb_model.fit(X_train, y_train)
pred=xgb_model.predict(X_validate)

from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_validate, pred)
print('MSE:', mse)

wmape=wmape(y_validate, pred)
print("WMAPE: {:.2%}".format(wmape))

MSE: 165873.69036960602
WMAPE: 35.09%


In [166]:
y_pred = xgb_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print('MSE:', mse)

wmape=wmape(y_test, y_pred)
print("WMAPE: {:.2%}".format(wmape))

[206.13737 206.13737 206.13737]
MSE: 22150.86500222818
WMAPE: 259.54%


# DecisionTreeRegressor

In [168]:
from sklearn.tree import DecisionTreeRegressor

DT_model=DecisionTreeRegressor()
DT_model.fit(X_train, y_train)

pred = DT_model.predict(X_validate)

from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_validate, pred)
print('MSE:', mse)

wmape=wmape(y_validate, pred)
print("WMAPE: {:.2%}".format(wmape))

MSE: 119433.0
WMAPE: 29.08%


In [170]:
y_pred = DT_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print('MSE:', mse)

wmape=wmape(y_test, y_pred)
print("WMAPE: {:.2%}".format(wmape))

MSE: 24867.0
WMAPE: 275.00%
